# Автогенерация текстовых описаний к видео (кейс Rutube)

В данном кейсе вам предлагается решить задачу автогенерации краткого тектового описания к видео, на основе видеофайла и автоматической транскрибации. 

Структура датасета следующая:

train.csv
- **video_name** - название видео (в директории **train_video**)
- **stt_name** - название файла с транскрибацией (в директории **train_stt**)
- **category_name** - категория видео
- **title** - название видео
- **description** - описание видео

В ноутбуке вы можете пронаблюдать baseline модель, без обучения (unsupervised) в качестве простого примера, основанную только на файле транскрибации. Также в конце считается метрика meteor по baseline модели и модели, которая из транскрипта речи (STT) выдает первые несколько предложений для сравнения. 

Тестовый датасет будет прислан вам позднее, поэтому здесь он фигурировать не будет.

Немного про модель: LexRankSummarizer, не вдаваясь в детали, можно сказать, что модель основана на статистиках, ее цель - найти самые "важные" предложения из полного текста (STT). 

Предложения представляются в виде мешка слов и получают эмбеддинги c tfidf, далее считаются косинусные близости предложений друг с другом. Следующая часть модели взята из немалоизвестной PageRank - строится граф, где на рёбрах стоит косинусная близость. Финальная часть  - по графу строится матрица, в ней находится максимальное сингулярное значение и таким образом находятся самые "значимые" предложения из большого текста.

Подробнее можно почитать например тут https://www.codingninjas.com/studio/library/lexrank

На метрики и сравнение моделей на других бенчмарках тут https://www.dialog-21.ru/media/5764/golovizninavspluskotelnikovev038.pdf

Для предобработки данных мы только удаляем стоп-слова (слишком часто встречаемые, например предлоги, союзы и тп), которые могут портить модель.

In [1]:
# lex rank - unsupervised upproach
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
import nltk
from nltk.corpus import stopwords
import numpy as np


# nltk.download('stopwords')

# Допольнительные стоп-слова можно скачать здесь
# https://github.com/stopwords-iso/stopwords-ru/blob/master/raw/stop-words-russian.txt
# но в этот список мы также добавили пару примеров вручную, поэтому прикладываем готовый файл. 
# Вы также можете модифицировать на свое усмотрение, или вовсе от него отказаться

with open("stop-words-russian.txt", 'r') as f:
    extra_stop_words = f.readlines()
    extra_stop_words = [line.strip() for line in extra_stop_words]


def sumy_method(text, n_sent: int = 4):
    
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    
    stemmer = Stemmer("russian")
    summarizer = LexRankSummarizer(stemmer)
    stopwords_ru = stopwords.words('russian')
    stopwords_ru.extend(extra_stop_words)
    summarizer.stop_words = stopwords_ru
    
    #Summarize the document with n_sent sentences
    summary = summarizer(parser.document, n_sent)
    dp = []
    if len(summary)> 0:
        for i in summary:
            lp = str(i)
            dp.append(lp)
    
        final_sentence = ' '.join(dp)
    else:
        final_sentence = ''
    if len(final_sentence.split(" "))>512:
        final_sentence = " ".join(final_sentence.split(" ")[:512])
    return final_sentence

In [2]:
import pandas as pd
import os
PATH_TO_DATA = 'train/'
dataset = pd.read_csv(os.path.join(PATH_TO_DATA, "train.csv"))

In [3]:
dataset.head(5)

,video_name,stt_name,category_name,title,description
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку..."
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...


### Для части видео речи может не быть, в бейзлайне мы это никак не учитываем, но вам предлагаем поработать и с такими ситуациями
31.mp4, 74.mp4, 111.mp4, 298.mp4, 478.mp4 - нет речи


In [4]:
dataset[dataset.video_name == '478.mp4']

,video_name,stt_name,category_name,title,description
478,478.mp4,478.txt,Путешествия,Прогулка по стране - Екатеринбург,Прогулка по Екатеринбургу — третьему по величи...


In [5]:
with open(os.path.join(PATH_TO_DATA, 'train_stt', '478.txt'), 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
lines

[]

### Чтобы понять сколько предложений нам нужно выдавать в качестве описания, посчитаем статистики

In [6]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/kivanova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
dataset['len'] = dataset.description.apply(lambda l : len(sent_tokenize(l)))
print("Среднее число предложений в трейн датасете", np.mean(dataset['len'].to_list()))
print("Медиана", np.median(dataset['len'].to_list()))

Среднее число предложений в трейн датасете 4.032
Медиана 3.0


### Теперь поймём примерный размер в токенах

In [8]:
dataset['len_tokens'] = dataset.description.apply(lambda l : len(l.split(" ")))

print("Среднее число слов в трейн датасете", np.mean(dataset['len_tokens'].to_list()))
print("Медиана", np.median(dataset['len_tokens'].to_list()))
print("Максимум", np.max(dataset['len_tokens'].to_list()))

Среднее число слов в трейн датасете 51.324
Медиана 42.0
Максимум 348


In [9]:
# поэтому в sumy_method мы добавили ограничение на число слов в сгенерированном тексте 
# (512 слов в нашем случае, решили так ограничить макс 348 слов из трейна)

### Генерируем текстовые описания для всех видео из трейна по текстовому описанию (из Speech To Text)
Если в видео не было речи, то в качестве описания ставим категорию видео

In [10]:
# Очистим STT от временных кодов
from tqdm import tqdm
tqdm.pandas()
def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

In [11]:
def gen_description(stt_name, n_sent, category_name):

    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = sumy_method(lines, n_sent)
        if len(res)>0:
            return res
        else:
            return category_name


In [12]:
%%time
dataset['stt_sum'] = np.nan
dataset['stt_sum'] = dataset.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 500/500 [06:32<00:00,  1.27it/s]

CPU times: user 8min 2s, sys: 5min 8s, total: 13min 10s
Wall time: 6min 32s


In [13]:
# видео, по которым нет речи и соответсвенно модель не смогла ничего выдать
dataset[dataset.stt_sum.isin(dataset.category_name.unique())]

,video_name,stt_name,category_name,title,description,len,len_tokens,stt_sum
31,31.mp4,31.txt,Авто-мото,DSC OFF на Байкальской миле 2020,"ГАЗ-24 «Волга КГБ» — проект Гурама Инцкирвели,...",4,48,Авто-мото
111,111.mp4,111.txt,Спорт,Команда MOTORCITY на Байкальской миле 2020,MOTORCITY собрала на фестиваль скорости “Байка...,5,116,Спорт
298,298.mp4,298.txt,Авто-мото,IVECO грузовик на Байкальской миле 2020,"Фестиваль скорости ""Байкальская Миля 2020"" соб...",4,47,Авто-мото
478,478.mp4,478.txt,Путешествия,Прогулка по стране - Екатеринбург,Прогулка по Екатеринбургу — третьему по величи...,2,23,Путешествия


In [14]:
dataset.head(3)

,video_name,stt_name,category_name,title,description,len,len_tokens,stt_sum
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,3,54,"Итак, после первого раунда третье место с резу..."
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,4,48,"Самое, наверное, быстрое, чтобы мы сейчас не п..."
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...",5,60,"Короче, Авет, сколько всего произошло на прошл..."


In [15]:
dataset.stt_sum.to_list()[:3]

['Итак, после первого раунда третье место с результатом два балла у нас занимает Кика. Картина стоит в долларах, не в рублях. Давай, делаем вторую картину. Ну, правильно боишься, потому что цена данной картины составляет 1 тысяча долларов.',
 'Самое, наверное, быстрое, чтобы мы сейчас не потеряли отрыв, это включить печку на максимальную жаркую температуру, закрыть все окна и продолжать ехать так до конца поездки. В смысле, печка на максимум, задание выполнено, мы едем. Что с Аветом, я не понимаю, он грузит видос уже где-то минут 25 и не может загрузить. Авет написал, что мы в скорой, Тёма ударился головой, интернет не грузит, мне пи... пацаны.',
 'Короче, Авет, сколько всего произошло на прошлой неделе ты даже не в курсе. Потому что у тебя на лице и так черные точки. И знаешь, Черное море подходит, кубернатор Краснодарского края говорит, море, у тебя есть QR-код? А зачем мы это смотрим?']

In [16]:
# dataset.to_csv("train_with_generated_sum.csv")

### Посчитаем метрику meteor

In [17]:
def func(stt_name, text, text_sum):
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0
dataset['meteor'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.stt_sum), axis=1)

In [18]:
print("Значение метрики meteor для unsupervised модели", dataset.meteor.mean())

Значение метрики meteor для unsupervised модели 0.0997954


In [19]:
# метрика в данной реализации имеет значения от 0 до 1

### Сравним с моделью, которая выдает первые 4 предложения из STT

In [26]:
%%time
def func(stt_name, text, category_name):
    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        res = lines[:4]
    res = " ".join(lines)
    if isinstance(res, str):
        return round(meteor([word_tokenize(text)],word_tokenize(res)), 4)
    else:
        return round(meteor([word_tokenize(text)],word_tokenize(category_name)), 4)
dataset['meteor_first4'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.category_name), axis=1)

CPU times: user 45 s, sys: 12.2 ms, total: 45 s
Wall time: 45 s


In [27]:
print("Значение метрики meteor для модели, выдающей первые 4 предложения", dataset.meteor_first4.mean())

Значение метрики meteor для модели, выдающей первые 4 предложения 0.057508199999999995


### Даже если вы не будете работать с видео контентом, способов делать суммаризацию текста - масса. В целом методы делятся на экстрактивные и абстрактивные (extractive/abstractive). Наш пример - экстрактивный. К асбтрактивным относятся всяческие языковые модели, например T5, можете посмотреть примеры, как его тьюнить: 
- https://github.com/priya-dwivedi/Deep-Learning/blob/master/wikihow-fine-tuning-T5/Tune_T5_WikiHow-Github.ipynb

Путём навешивания дополнительных слоёв, можно затьюнить BERT на абстрактивный способ генерации текста
- https://github.com/nlpyang/PreSumm/tree/master

Также можно затьюнить BERT на экстрактивный подход
- https://github.com/chriskhanhtran/bert-extractive-summarization/tree/master



### Но не забывайте, что для части видео речи нет и иногда речь не всегда соотвествует видеоматериалу, поэтому стоит обратить внимание на Image/Video captioning
- https://aclanthology.org/2020.emnlp-main.61.pdf